# Deliverable 8
Breitzman wants:
Quarterly Snapshots By Region

In [1]:
import os
from enum import Enum
import csv
import sqlite3 as lite
from decimal import Decimal
from datetime import date, datetime, timedelta

In [2]:
FILE_PATHS = {
    'DB_TEAM_8' : './../0_SD_Team_8/store_team_8.db',
    'DB_TEAM_9' : './../0_SD_Team_9/grocery_store.db',
    'DB_TEAM_10' : './../0_SD_Team_10/grocery_team_10_v2.db',
    'PRODUCTS_CSV' : './../2_Product_Mapping/PRODUCTS_MAPPED.csv'
}

DATA_MART_PATH = './../A_Build_Data_Mart/Region_C_Data_Mart.db'

In [3]:
file_exists = os.path.isfile(DATA_MART_PATH)

if(file_exists):
    print(f'OK - \'{DATA_MART_PATH}\'')
else:
    print(f'MISSING - \'{DATA_MART_PATH}\'')
    raise SystemExit('\n' "ERROR!" '\n' "You are missing files!" '\n' "Read and Follow the Cell instructions provided.")

OK - './../A_Build_Data_Mart/Region_C_Data_Mart.db'


In [4]:
'''
    If I try to make db_options an inner class to db, 
        I get an error saying that the class is undefined.
'''
class db_options(Enum):
        DEFAULT = 0
        RETURN_RESULTS = 1
        PRINT_RESULTS = 2

class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql, values)


    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

In [5]:
DB_HANDLE = db(DATA_MART_PATH)

In [6]:
DB_HANDLE.connect()

results = DB_HANDLE.execute_sql('SELECT * FROM inventory_quarterly WHERE Quarter = 1 ORDER BY StoreKey, ProductKey', options=db_options.RETURN_RESULTS)

column_names = []

for item in DB_HANDLE.cur.description:
    column_names.append(item[0])

FILE_NAME = 'Deliverable_8_Q1_Snapshot.txt'

try:
    os.delete(FILE_NAME)
except:
    # Do nothing
    pass

with open(FILE_NAME, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(column_names)

    for row in results:
        writer.writerow(row)

DB_HANDLE.close()